# An Improved VAE

## Table of Contents
1. [Data and Model Definitions](Data-and-Model-Definitions)
2. [Basic Model Analysis](#Basic-Model-Analysis)
3. [Training with New Genres](#Training-with-New-Genres)
4. [Training at Scale](#Training-at-Scale)

In [1]:
import librosa, os #audio processing and file system parsing
import librosa.display
import numpy as np #math library
import tensorflow as tf #for model building
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Lambda, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt # for visualization
import pandas as pd #for data analysis / prep
import IPython.display as ipd #for sound output
from scipy.spatial.distance import cosine

2025-03-14 21:04:52.867874: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-14 21:04:52.899145: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 21:04:53.176396: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-14 21:04:53.406422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742000693.602207   24919 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742000693.65

## Data and Model Definitions

In this section we define our core data processing techniques and VAE model structure. Our data processing techniques are a huge improvement from `notebooks/simple_vae.ipynb` because we unnormalize our samples and also pad/truncate the audio data.

In [2]:
# Define hop_length (adjust as needed)
HOP_LENGTH = 512
TIME_FRAMES = 1290

def load_data(audio_dir: str, n_mels: int=128, time_frames: int=216):
    """Processes a directory of audio files into a DataFrame which contains the ground truth audio, ground truth spectrogram, and normalization parameters min_max for each song."""
    audio_files = os.listdir(audio_dir)
    audio_data = {'filename': [], 'gt_audio': [], 'gt_spectrogram': [], 'min_max': []}

    for file in audio_files:
        try:
            original_audio, sampling_rate = librosa.load(os.path.join(audio_dir, file), sr=None)  # Preserve original sampling rate

            mel_spectrogram = librosa.feature.melspectrogram(y=original_audio, sr=sampling_rate, n_mels=n_mels, hop_length=HOP_LENGTH)
            S_dB = librosa.power_to_db(mel_spectrogram, ref=np.max)  # Normalize based on max power
            
            # Save min/max values for proper denormalization later
            S_dB_min = np.min(S_dB)
            S_dB_max = np.max(S_dB)
            
            # Normalize to [0,1]
            S_dB = (S_dB - S_dB_min) / (S_dB_max - S_dB_min)

            # **Truncate or pad spectrogram to $time frames**
            if S_dB.shape[1] < TIME_FRAMES:
                pad_width = TIME_FRAMES - S_dB.shape[1]
                S_dB = np.pad(S_dB, pad_width=((0, 0), (0, pad_width)), mode='constant')
            elif S_dB.shape[1] > TIME_FRAMES:
                S_dB = S_dB[:, :TIME_FRAMES]
            
            # Store data
            audio_data['filename'].append(file)
            audio_data['gt_audio'].append(original_audio)
            audio_data['gt_spectrogram'].append(S_dB)
            audio_data['min_max'].append((S_dB_min, S_dB_max))  # Save for denormalization
        
        except Exception as e:
            print(f"Skipping file {file} due to error: {e}")
    
    return pd.DataFrame(audio_data)

def reconstruct_audio(spectrogram, sr=22050, n_mels=128, min_max=None):
    """Reconstructs audio from a normalized Mel spectrogram. Utilizes parameters from normalization to reverse process!"""
    spectrogram = np.array(spectrogram)
    S_dB_min, S_dB_max = min_max
    S_dB = spectrogram * (S_dB_max - S_dB_min) + S_dB_min  # Reverse normalization

    # Convert back to power spectrogram (ensure this transformation is correct)
    S = librosa.db_to_power(S_dB)  

    # Convert Mel spectrogram back to waveform
    reconstructed_audio = librosa.feature.inverse.mel_to_audio(S, sr=sr, hop_length=HOP_LENGTH)  
    return reconstructed_audio


In [3]:
jazz_df = load_data("../data/kaggle/genres_original/jazz/")
jazz_df

/tmp/ipykernel_24919/1707909223.py:12: UserWarning: PySoundFile failed. Trying audioread instead.
  original_audio, sampling_rate = librosa.load(os.path.join(audio_dir, file), sr=None)  # Preserve original sampling rate
/home/seansteinle/.pyenv/versions/3.10.16/envs/cs757-p1/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Skipping file jazz.00054.wav due to error: 


,filename,gt_audio,gt_spectrogram,min_max
0,jazz.00080.wav,"[-0.010284424, -0.018707275, -0.014312744, -0....","[[0.6116525, 0.6273083, 0.59515595, 0.59347874...","(-62.807693, 0.0)"
1,jazz.00098.wav,"[-0.031066895, -0.05078125, -0.04537964, -0.04...","[[0.6259705, 0.5301552, 0.46367303, 0.4521164,...","(-80.0, 0.0)"
2,jazz.00001.wav,"[0.0024108887, 0.005493164, 0.008666992, 0.011...","[[0.59155864, 0.52710944, 0.507942, 0.5584143,...","(-80.0, 0.0)"
3,jazz.00069.wav,"[-0.029541016, -0.07070923, -0.10971069, -0.13...","[[0.65641576, 0.57051414, 0.40731898, 0.422603...","(-76.149796, 0.0)"
4,jazz.00050.wav,"[-0.13595581, -0.09390259, -0.06210327, -0.216...","[[0.842084, 0.87873775, 0.8421996, 0.8063969, ...","(-72.79858, 0.0)"
...,...,...,...,...
94,jazz.00008.wav,"[0.056854248, 0.075653076, 0.040283203, 0.0259...","[[0.54276, 0.5168449, 0.50455403, 0.6284218, 0...","(-80.0, 0.0)"
95,jazz.00013.wav,"[0.02557373, 0.043823242, 0.036010742, 0.02526...","[[0.6091228, 0.63428605, 0.6647167, 0.7280141,...","(-80.0, 0.0)"
96,jazz.00095.wav,"[-0.062194824, -0.09188843, -0.072052, -0.0799...","[[0.56795484, 0.53531694, 0.5111699, 0.5515669...","(-80.0, 0.0)"
97,jazz.00097.wav,"[0.02279663, 0.03564453, 0.029205322, 0.029632...","[[0.475236, 0.4526332, 0.4655529, 0.47875708, ...","(-80.0, 0.0)"


In [4]:
def create_vae():
    #let's try an architecture modeled off of this article (https://yuehan-z.medium.com/how-to-train-your-ml-models-music-generation-try-it-out-d4c0ab01c9f4)
    n_mels, time_frames = 128, TIME_FRAMES #audio params
    latent_dim = 16 #dimensionality of VAE
    
    # Encoder
    encoder_input = Input(shape=(n_mels, time_frames))
    encoder_flatten = Flatten()(encoder_input)
    z_mean = Dense(latent_dim)(encoder_flatten)
    z_log_var = Dense(latent_dim)(encoder_flatten)
    
    def sampling(args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
    encoder = Model(encoder_input, [z_mean, z_log_var, z], name="encoder")
    
    #Decoder
    decoder_input = Input(shape=(latent_dim,))
    decoder_dense = Dense(n_mels * time_frames, activation="sigmoid")(decoder_input)
    decoder_reshaped = Reshape((n_mels, time_frames))(decoder_dense)
    decoder = Model(decoder_input, decoder_reshaped, name="decoder")
    
    #Loss
    def vae_loss_function(args):
        y_true, y_pred, z_mean, z_log_var = args
        reconstruction_loss = K.mean(K.square(y_true - y_pred))
        kl_loss = -0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))
        return reconstruction_loss + kl_loss
    
    #Full VAE
    # Create the VAE
    vae_input = Input(shape=(n_mels, time_frames))
    vae_target = Input(shape=(n_mels, time_frames))
    encoder_output = encoder(vae_input)
    decoder_output = decoder(encoder_output[2])
    
    # Add the loss function to the model using a Lambda layer
    loss = Lambda(vae_loss_function, output_shape=(1,), name='loss')([vae_target, decoder_output, encoder_output[0], encoder_output[1]])
    vae = Model(inputs=[vae_input, vae_target], outputs=[decoder_output, loss])

    return vae

In [5]:
#TRAIN VAE

#prep data
len(set(jazz_df['gt_spectrogram'].apply(lambda spec: spec.shape))) == 1 #all specs the same length?
jazz_songs = np.array(jazz_df['gt_spectrogram'].tolist()) #transform song shapes

#hyperparams
batch_size = 10
epochs = 100

# Compile the VAE
vae = create_vae()
vae.compile(optimizer=Adam(learning_rate=0.001), loss=['mse', None])

# Train the VAE
history = vae.fit([jazz_songs,jazz_songs], [jazz_songs,np.zeros_like(jazz_songs)],
                  epochs=epochs,
                  batch_size=batch_size)

2025-03-14 21:05:11.966379: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.0468
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0227 
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0190 
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0183 
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0167 
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 0.0165 
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0159 
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0152 
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0151 
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0160 
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0151 
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0146 
Epoch 13/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0137 
Epoch 14/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0140 
Epoch 15/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 8

## Basic Model Analysis

In [ ]:
pred_spectrograms, _ = vae.predict([jazz_songs, jazz_songs])
jazz_df['pred_spectrogram'] = pd.Series(pred_spectrograms.tolist())

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step 


In [ ]:
jazz_df

In [ ]:
def compute_cosine_similarity(df: pd.DataFrame, col1: str, col2: str):
    """
    Compute cosine similarity between two spectrogram columns in a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing spectrogram data.
        col1 (str): Name of the first column (e.g., ground truth spectrograms).
        col2 (str): Name of the second column (e.g., generated spectrograms).

    Returns:
        np.ndarray: Array of cosine similarities.
    """
    similarities = []
    
    for idx, row in df.iterrows():
        spec1 = np.array(row[col1]).flatten()  # Flatten to 1D
        spec2 = np.array(row[col2]).flatten()  # Flatten to 1D
        
        if spec1.shape != spec2.shape:
            print(f"Warning: Shape mismatch at index {idx} ({spec1.shape} vs {spec2.shape})")
            similarities.append(np.nan)
            continue

        similarities.append(1 - cosine(spec1, spec2))  # Cosine similarity (1 - cosine distance)

    return np.array(similarities)

def compare_song_audios(jazz_df: pd.DataFrame, sample_n: int=0):
    """Convenience function to play a single generated song."""
    pred_audio = reconstruct_audio(jazz_df.iloc[sample_n]['pred_spectrogram'], 22050, 128, jazz_df.iloc[sample_n]['min_max'])
    return ipd.Audio(jazz_df.iloc[sample_n]['gt_audio'], rate=22050),ipd.Audio(pred_audio, rate=22050)

# Visualize the original and reconstructed spectrograms
def plot_spectrogram_comparison(jazz_df: pd.DataFrame, index: int):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Plot the original spectrogram
    axes[0].imshow(jazz_df.iloc[index]['gt_spectrogram'], origin='lower', aspect='auto')
    axes[0].set_title("Original Spectrogram")

    # Plot the reconstructed spectrogram
    axes[1].imshow(jazz_df.iloc[index]['pred_spectrogram'], origin='lower', aspect='auto')
    axes[1].set_title("Reconstructed Spectrogram")

    plt.show()

Cosine Similarity Comparison

In [ ]:
jazz_df['cosine_similarity'] = compute_cosine_similarity(jazz_df, 'gt_spectrogram', 'pred_spectrogram')
jazz_df['cosine_similarity'].sort_values()
#looks like 51 is the worst reconstruction and 80 is the best!

Audio Comparison

In [ ]:
#worst reconstruction!
real, pred = compare_song_audios(jazz_df, 51)

In [ ]:
real

In [ ]:
pred

In [ ]:
#best reconstruction!
real, pred = compare_song_audios(jazz_df, 80)

In [ ]:
real

In [ ]:
pred

Obviously these results are disappointing, but remember two things:
1) We're not directly learning audio. We're learning spectrograms!
2) Audio>Spectrogram>Audio is an inherently lossy process. If you're not convinced, consider the loss even with a ground truth sample here.

Instead of judging entirely by audio, let's take a look at the spectograms we produced!

In [ ]:
sample_n = 80
original_song = jazz_df['gt_audio'][sample_n]
original_audio = ipd.Audio(original_song, rate=22050)
lossy_song = reconstruct_audio(jazz_df['gt_spectrogram'][sample_n], 22050, 128, jazz_df['min_max'][sample_n])
lossy_audio = ipd.Audio(lossy_song, rate=22050) #this sounds HORRIBLE! it is probably the fact that we didn't unnormalize?
print(f"original size: {original_song.shape}\ttransformed size: {lossy_song.shape}")

In [ ]:
original_audio

In [ ]:
lossy_audio

Spectrogram Analysis

In [ ]:
plot_spectrogram_comparison(jazz_df, 51)

This is our worst reconstructed sample by cosine similarity, and it's obvious why! We're missing a ton of notes and things just sort of smear together.

In [ ]:
plot_spectrogram_comparison(jazz_df, 80)

This is our best one and woah! It's so clear to tell!

## Training with New Genres

## Training at Scale

We've seen that our model can effectively learn new types of genres--but could it learn to produce all types of music at an even higher quality if we just feed it more training data? To find out, we increased our training set by a factor of 8 with the [Free Music Archive's](https://github.com/mdeff/fma) smallest dataset of 8,000 songs.

In [ ]:
def load_df_from_fma(music_dir: str, max_songs: int=8000):
    """This function loads a directory of songs and creates a simple dataframe with (song,numpy) rows, based on the FMA dataset. We expect music_dir to be a single-level directory with just songs."""
    audio_data,bad_paths = [],[]
    for i,song in enumerate(os.listdir(music_dir)):
        song_path = music_dir+song
        try:
            original_audio, sampling_rate = librosa.load(os.path.join(audio_dir, file), sr=None)  # Preserve original sampling rate
            
            mel_spectrogram = librosa.feature.melspectrogram(y=original_audio, sr=sampling_rate, n_mels=n_mels, hop_length=HOP_LENGTH)
            S_dB = librosa.power_to_db(mel_spectrogram, ref=np.max)  # Normalize based on max power
            
            # Save min/max values for proper denormalization later
            S_dB_min = np.min(S_dB)
            S_dB_max = np.max(S_dB)
            
            # Normalize to [0,1]
            S_dB = (S_dB - S_dB_min) / (S_dB_max - S_dB_min)

            # **Truncate or pad spectrogram to $time frames**
            if S_dB.shape[1] < TIME_FRAMES:
                pad_width = TIME_FRAMES - S_dB.shape[1]
                S_dB = np.pad(S_dB, pad_width=((0, 0), (0, pad_width)), mode='constant')
            elif S_dB.shape[1] > TIME_FRAMES:
                S_dB = S_dB[:, :TIME_FRAMES]
                
            # Store data
            audio_data['filename'].append(file)
            audio_data['gt_audio'].append(original_audio)
            audio_data['gt_spectrogram'].append(S_dB)
            audio_data['min_max'].append((S_dB_min, S_dB_max))  # Save for denormalization
            
        except Exception as e:
            print(f"couldn't load: {song_path}, got: {e}")
            bad_paths.append([song_path,e])
        if i >= max_songs:
            break
    return pd.DataFrame(music_dicts)

In [ ]:
fma_df = load_data('../data/fma/small/')

In [ ]:
vae = create_vae()
vae